In [2]:
!pip install pydub

## Audio Slicing


- Input_folder : 내가 자를 wav 파일들 있는 폴더 경로
- Output_folder : 내가 자른거 저장할 wav 파일들 있는 폴더 경로

In [102]:
import os
import re
from pydub import AudioSegment
from tqdm import tqdm

def get_file_paths(directory):
    file_paths = []

    # directory 내의 모든 파일 및 디렉터리 목록을 가져옵니다.
    for file_name in os.listdir(directory):
        # .DS_Store 파일은 무시
        if file_name == '.DS_Store':
            continue
            
        # 파일의 전체 경로를 생성합니다.
        full_path = os.path.join(directory, file_name)

        # 경로가 파일인지 확인하고 리스트에 추가합니다.
        if os.path.isfile(full_path):
            file_paths.append(full_path)

    return file_paths

##### 15초로 자르고, 15초 미만은 그냥 저장하는 함수 선언

In [103]:
# 파일에서 오디오를 15초씩 자르고 저장하는 함수 정의
def split_audio(file_path, output_folder):
    audio = AudioSegment.from_wav(file_path)
    duration = len(audio) / 1000  # 오디오 길이(초)로 변환
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    if duration < 15:
        # 파일이 15초 미만인 경우 그대로 저장
        output_path = os.path.join(output_folder, f"{file_name}_1.wav")
        audio.export(output_path, format="wav")
        print(f"Saved: {output_path}")
        return

    # 15초씩 오디오 파일 자르기
    part = 1
    for i in range(0, len(audio), 15000):
        start = i
        end = i + 15000
        chunk = audio[start:end]

        chunk_filename = os.path.join(output_folder, f"{file_name}_{part}.wav")
        chunk.export(chunk_filename, format="wav")
        print(f"Saved: {chunk_filename}")
        part += 1

##### 15초로 자르고, 10초 미만이면 제거하는 함수 선언

In [104]:
import os
from pydub import AudioSegment

def shorter_than_10s(audio_segment):
    duration = len(audio_segment) / 1000  # 오디오 길이(초)로 변환
    return duration < 10

# 파일에서 오디오를 15초씩 자르고 저장하는 함수 정의
def split_audio_15(file_path, output_folder):
    try:
        audio = AudioSegment.from_wav(file_path)
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return
    
    file_name = os.path.splitext(os.path.basename(file_path))[0]

    # 15초씩 오디오 파일 자르기
    part = 1
    for i in range(0, len(audio), 15000):
        start = i
        end = i + 15000
        chunk = audio[start:end]
        
        # 15초로 슬라이스하고 남은 파일이 3초 이하면 저장하지 않기
        if shorter_than_10s(chunk):
            continue
        
        chunk_filename = os.path.join(output_folder, f"{file_name}_{part}.wav")
        chunk.export(chunk_filename, format="wav")
        print(f"Saved: {chunk_filename}")
        part += 1


##### audio slicing 적용

In [105]:
# 입력 및 출력 폴더 경로 설정
input_folder = 'STS1/news/wav'
output_folder = 'STS1/10s_news/wav'

# 출력 폴더가 존재하지 않으면 생성
os.makedirs(output_folder, exist_ok=True)

In [106]:
wav_paths = get_file_paths(input_folder)

In [107]:
# wav_paths 목록의 모든 파일 처리
for file_path in tqdm(wav_paths, desc='Processing files'):
    split_audio_15(file_path, output_folder)

Processing files: 100%|████████████████████████| 76/76 [00:00<00:00, 981.60it/s]

Saved: STS1/10s_news2/wav/AI_1_택시기사 폭행 한지선 결국 출연 드라마 하차  연합뉴스 (Yonhapnews)_5_1.wav
Saved: STS1/10s_news2/wav/AI_1_BBC 아바 노래인 줄…김정은 찬양 북한 뮤비 틱톡 인기 연합뉴스 (Yonhapnews)_11_1.wav
Saved: STS1/10s_news2/wav/AI_1_LG V50 씽큐 출시 5G 가입자 쟁탈전…9만원 요금제 쓰면 0원  연합뉴스 (Yonhapnews)_1_1.wav
Saved: STS1/10s_news2/wav/AI_1_트럼프 미국 대통령 내달 G20 계기 방한…靑 브리핑  연합뉴스 (Yonhapnews)_3_1.wav
Saved: STS1/10s_news2/wav/AI_1_한센병 발언 논란 김현아 의원 하루 만에 사과  연합뉴스 (Yonhapnews)_4_1.wav
Saved: STS1/10s_news2/wav/AI_1_트럼프 중국 보복 상대적으로 매우 크진 않아…자신감 피력  연합뉴스 (Yonhapnews)_2_1.wav
Saved: STS1/10s_news2/wav/AI_1_트럼프 12만 중동 파병설 부인…보낸다면 더 많이 보낼 것  연합뉴스 (Yonhapnews)_1_1.wav
Saved: STS1/10s_news2/wav/AI_1_패션쇼엔 열광 가자전쟁엔 침묵…디지털단두대 오른 미 셀럽 연합뉴스 (Yonhapnews)_3_1

In [108]:
# wav 파일 RIFF header 제대로 있는지 확인
wav_path = 'STS1/10s_news2/wav/AI_1_택시기사 폭행 한지선 결국 출연 드라마 하차  연합뉴스 (Yonhapnews)_5_1.wav'

with open(wav_path, "rb") as f:
    data = f.read()
data[:30]

b'RIFF$S\x07\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01\x00\x80>\x00\x00\x00}'